Machine learning pipeline refers to the complete workflow and processes of building and deploying a machine learning model. It automates and standardizes the workflow involved in creating a machine-learning model.

A machine learning pipeline consists of sequential steps, which include preprocessing and  model training to deployment.
so, we need follow these steps:
1. Load data.
2. Data Preprocessing.
3. Data cleaning and scaling.
4. Feature Engineering.
5. Model training.
6. Prediction Generation.

In [ ]:
#import libraries
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
#Machine learning algorithms
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
pd.set_option('display.max_columns', None)

now first we need clearly define the features needed for both models

- For classification model:

[  'MonthsDelinquent', 'FICO_Category_Encoded',
                            'YearsInRepayment_Group_Encoded', 'NumBorrowers',
                            'Channel_Frequency', 'LTV_Group_Encoded','OCLTV_Group_Encoded','isHomeFirstBuyer',
                            'OrigInterestRate']

- For Regression model:

regression_features = ['OrigUPB', 'MonthlyPayment', 'total_amount',
                       'interest_amount', 'MonthlyIncome', 'DTIBins']

the unssen data should be like this raw data

In [ ]:
raw_data=pd.read_csv('/content/drive/MyDrive/Prepayment-Mortgage-Trading-Analysis-and-Prediction/MyNewdata.csv')

<ipython-input-39-083f17cdbe04>:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data=pd.read_csv('/content/drive/MyDrive/Prepayment-Mortgage-Trading-Analysis-and-Prediction/MyNewdata.csv')


In [ ]:
raw_data.isna().sum()

,0
CreditScore,0
FirstPaymentDate,0
FirstTimeHomebuyer,0
MaturityDate,0
MSA,0
MIP,0
Units,0
Occupancy,0
OCLTV,0
DTI,0


In [ ]:
raw_data.columns

Index(['CreditScore', 'FirstPaymentDate', 'FirstTimeHomebuyer', 'MaturityDate',
       'MSA', 'MIP', 'Units', 'Occupancy', 'OCLTV', 'DTI', 'OrigUPB', 'LTV',
       'OrigInterestRate', 'Channel', 'PPM', 'ProductType', 'PropertyState',
       'PropertyType', 'PostalCode', 'LoanSeqNum', 'LoanPurpose',
       'OrigLoanTerm', 'NumBorrowers', 'SellerName', 'ServicerName',
       'EverDelinquent', 'MonthsDelinquent', 'MonthsInRepayment'],
      dtype='object')

In [ ]:
# Define the bins and labels
bins = [0, 80, 90, 100]
labels = ['Low LTV', 'Medium LTV', 'High LTV']

# Create a new column 'LTV_Group' with the grouped values
raw_data['LTV_range'] = pd.cut(raw_data['LTV'], bins=bins, labels=labels, right=False)

In [ ]:
# Define the bins and labels
bins = [0, 80, 90, 100]
labels = ['Low LTV', 'Medium LTV', 'High LTV']

# Create a new column 'LTV_Group' with the grouped values
raw_data['OCLTV_range'] = pd.cut(raw_data['LTV'], bins=bins, labels=labels, right=False)

In [ ]:
# Step 1: Remove rows where 'CreditScore' is 0
raw_data = raw_data[raw_data['CreditScore'] != 0]

In [ ]:
# Define the bins and labels for FICO Score Categories
fico_bins = [300, 580, 670, 740, 800, 851]
fico_labels = ['Very Poor', 'Poor', 'Fair', 'Good', 'Excellent']
# Create a new column 'FICO_Category' with the grouped values
raw_data['Credit_range'] = pd.cut(raw_data['CreditScore'], bins=fico_bins, labels=fico_labels, right=False)

In [ ]:
# Convert MonthsInRepayment to years
raw_data['YearsInRepayment'] = raw_data['MonthsInRepayment'] / 12

# Display the updated DataFrame
raw_data[['MonthsInRepayment', 'YearsInRepayment']].head(20)

,MonthsInRepayment,YearsInRepayment
1078,163,13.583333
1079,165,13.750000
1080,43,3.583333
1081,65,5.416667
1082,42,3.500000
1083,55,4.583333
1084,39,3.250000
1085,133,11.083333
1086,53,4.416667
1087,64,5.333333


In [ ]:
# Define the bins and labels for YearsInRepayment categories
bins = [0, 4, 8, 12, 16, 20, float('inf')]
labels = ['0-4 years', '4-8 years', '8-12 years', '12-16 years', '16-20 years', '20+ years']

# Create a new column 'YearsInRepayment_Group' with the grouped values
raw_data['Repay_range'] = pd.cut(
    raw_data['YearsInRepayment'], bins=bins, labels=labels, right=False, include_lowest=True
)

# Display the updated DataFrame
raw_data[['YearsInRepayment', 'Repay_range']].head(20)


,YearsInRepayment,Repay_range
1078,13.583333,12-16 years
1079,13.750000,12-16 years
1080,3.583333,0-4 years
1081,5.416667,4-8 years
1082,3.500000,0-4 years
1083,4.583333,4-8 years
1084,3.250000,0-4 years
1085,11.083333,8-12 years
1086,4.416667,4-8 years
1087,5.333333,4-8 years


Select the specific columns needed for model fiting

In [ ]:
raw_data_features=raw_data[["FirstTimeHomebuyer",
    "MIP",
    "Units",
    "OrigUPB",
    "OrigInterestRate",
    "OrigLoanTerm",
    "Channel",
    "LoanPurpose",
    "NumBorrowers",
    "SellerName",
    "ServicerName",
    "Credit_range",
    "LTV_range",
    "OCLTV_range",
    "DTI",
    "Repay_range",
    "MonthsDelinquent",'EverDelinquent']]

In [ ]:
raw_data_features

,FirstTimeHomebuyer,MIP,Units,OrigUPB,OrigInterestRate,OrigLoanTerm,Channel,LoanPurpose,NumBorrowers,SellerName,ServicerName,Credit_range,LTV_range,OCLTV_range,DTI,Repay_range,MonthsDelinquent,EverDelinquent
1078,N,0,1,116000,6.875,360,T,C,2,CO,Other servicers,Very Poor,Low LTV,Low LTV,21,12-16 years,0,0
1079,N,0,1,96000,6.875,360,R,N,1,Ot,Other servicers,Very Poor,Low LTV,Low LTV,28,12-16 years,1,1
1080,X,0,1,101000,6.750,360,R,N,2,Ot,Other servicers,Very Poor,Low LTV,Low LTV,18,0-4 years,0,0
1081,N,0,1,66000,6.625,360,R,C,2,GM,GMACMTGECORP,Very Poor,Low LTV,Low LTV,25,4-8 years,0,0
1082,X,0,1,234000,6.750,360,T,N,1,Ot,Other servicers,Very Poor,Medium LTV,Medium LTV,38,0-4 years,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266452,N,0,1,90000,7.000,360,R,N,2,Ot,Other servicers,Excellent,Low LTV,Low LTV,32,0-4 years,0,0
266453,N,0,1,267000,6.375,319,C,P,2,Ot,Other servicers,Excellent,Low LTV,Low LTV,0,4-8 years,0,0
266454,N,0,1,87000,5.625,313,R,N,1,Ot,Other servicers,Excellent,Low LTV,Low LTV,31,0-4 years,0,0
266455,N,0,1,88000,7.000,360,R,P,2,Ot,Other servicers,Excellent,Medium LTV,Medium LTV,54,0-4 years,0,0


In [ ]:
raw_data_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265379 entries, 1078 to 266456
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   FirstTimeHomebuyer  265379 non-null  object  
 1   MIP                 265379 non-null  int64   
 2   Units               265379 non-null  int64   
 3   OrigUPB             265379 non-null  int64   
 4   OrigInterestRate    265379 non-null  float64 
 5   OrigLoanTerm        265379 non-null  int64   
 6   Channel             265379 non-null  object  
 7   LoanPurpose         265379 non-null  object  
 8   NumBorrowers        265379 non-null  object  
 9   SellerName          265379 non-null  object  
 10  ServicerName        265379 non-null  object  
 11  Credit_range        265379 non-null  category
 12  LTV_range           265372 non-null  category
 13  OCLTV_range         265372 non-null  category
 14  DTI                 265379 non-null  int64   
 15  Repay_range        

In [ ]:
raw_data_features1=raw_data_features[0:150000] # for fit

In [ ]:
raw_data_features1

,FirstTimeHomebuyer,MIP,Units,OrigUPB,OrigInterestRate,OrigLoanTerm,Channel,LoanPurpose,NumBorrowers,SellerName,ServicerName,Credit_range,LTV_range,OCLTV_range,DTI,Repay_range,MonthsDelinquent,EverDelinquent
1078,N,0,1,116000,6.875,360,T,C,2,CO,Other servicers,Very Poor,Low LTV,Low LTV,21,12-16 years,0,0
1079,N,0,1,96000,6.875,360,R,N,1,Ot,Other servicers,Very Poor,Low LTV,Low LTV,28,12-16 years,1,1
1080,X,0,1,101000,6.750,360,R,N,2,Ot,Other servicers,Very Poor,Low LTV,Low LTV,18,0-4 years,0,0
1081,N,0,1,66000,6.625,360,R,C,2,GM,GMACMTGECORP,Very Poor,Low LTV,Low LTV,25,4-8 years,0,0
1082,X,0,1,234000,6.750,360,T,N,1,Ot,Other servicers,Very Poor,Medium LTV,Medium LTV,38,0-4 years,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151073,Y,30,1,101000,6.750,360,R,P,1,WA,WASHINGTONMUTUALBANK,Fair,High LTV,High LTV,38,4-8 years,0,0
151074,Y,30,1,112000,6.625,360,R,P,2,WA,WASHINGTONMUTUALBANK,Fair,High LTV,High LTV,36,0-4 years,0,0
151075,Y,30,1,120000,6.625,360,T,P,2,PR,PRINCIPALRESIDENTIAL,Fair,High LTV,High LTV,23,0-4 years,0,0
151076,Y,18,1,167000,7.000,360,T,P,2,FL,CHASEMANHATTANMTGECO,Fair,High LTV,High LTV,43,0-4 years,0,0


In [ ]:
raw_data_features2=raw_data_features[150000:200000] # for predict

In [ ]:
raw_data_features2

,FirstTimeHomebuyer,MIP,Units,OrigUPB,OrigInterestRate,OrigLoanTerm,Channel,LoanPurpose,NumBorrowers,SellerName,ServicerName,Credit_range,LTV_range,OCLTV_range,DTI,Repay_range,MonthsDelinquent,EverDelinquent
151078,Y,30,1,196000,6.875,360,R,P,2,AC,GMACMTGECORP,Fair,High LTV,High LTV,43,0-4 years,0,0
151079,Y,18,1,51000,7.375,360,T,P,1,NO,WELLSFARGOBANKNA,Fair,High LTV,High LTV,45,16-20 years,0,0
151080,Y,30,1,72000,7.000,360,T,P,1,FL,CHASEMANHATTANMTGECO,Fair,High LTV,High LTV,33,0-4 years,0,0
151081,Y,30,1,173000,7.500,360,R,P,2,CO,COUNTRYWIDE,Fair,High LTV,High LTV,29,4-8 years,0,0
151082,Y,0,1,99000,6.875,360,T,P,1,CO,COUNTRYWIDE,Fair,Low LTV,Low LTV,34,0-4 years,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201073,N,0,1,196000,6.875,360,T,P,2,FL,WASHINGTONMUTUALBANK,Good,Medium LTV,Medium LTV,42,0-4 years,0,0
201074,N,0,1,176000,6.875,360,T,P,1,FL,WASHINGTONMUTUALBANK,Good,Medium LTV,Medium LTV,45,4-8 years,1,1
201075,N,12,1,168000,7.125,360,T,N,2,FL,WASHINGTONMUTUALBANK,Good,Medium LTV,Medium LTV,42,0-4 years,0,0
201076,N,25,1,130000,7.375,360,R,P,2,BA,BANKOFAMERICANA,Good,High LTV,High LTV,46,0-4 years,1,1


In [ ]:
Target1=raw_data_features1[['EverDelinquent']]

In [ ]:
Target2=raw_data_features2[['EverDelinquent']]

In [ ]:
# Preprocessing transformer
class PreprocessingTransformer(BaseEstimator, TransformerMixin):
    def transform(self, X):
        def prepayment(dti,income):
            if (dti<40):
              p =income/2
            else:
              p=income*3/4
            return p
        # Preprocessing steps
        X = X.copy()

        # Handling 'FirstTimeHomebuyer'
        X = X[X['FirstTimeHomebuyer'] != 'X']
        X['isHomeFirstBuyer'] = X['FirstTimeHomebuyer'].map({'Y': 1, 'N': 0})

        # Handling 'NumBorrowers'
        X = X[X['NumBorrowers'] != 'X ']
        X['NumBorrowers'] = X['NumBorrowers'].astype(int)

        # Channel frequency encoding
        for col in ['Channel']:
            freq_encoding = X[col].value_counts(normalize=True)
            X[f'{col}_Frequency'] = X[col].map(freq_encoding)



        # Encode categorical features
        X['LTV_Group_Encoded'] = X['LTV_range'].map({'Low LTV': 0, 'Medium LTV': 1, 'High LTV': 2})
        X['OCLTV_Group_Encoded'] = X['OCLTV_range'].map({'Low LTV': 0, 'Medium LTV': 1, 'High LTV': 2})
        X['FICO_Category_Encoded'] = X['Credit_range'].map({'Very Poor': 0, 'Poor': 1, 'Fair': 2, 'Good': 3, 'Excellent': 4})
        X['YearsInRepayment_Group_Encoded'] = X['Repay_range'].map({'0-4 years': 0, '4-8 years': 1, '8-12 years': 2, '12-16 years': 3, '16-20 years': 4, '20+ years': 5})

        # DTI bins
        bins = [0, 20, 30, 40, 50, 100]
        labels = ['Low', 'Moderate', 'High', 'Very High', 'Extreme']
        X['DTIBin'] = pd.cut(X['DTI'], bins=bins, labels=labels, right=False)
        X['DTIBins'] = X['DTIBin'].map({'Low': 0, 'Moderate': 1, 'High': 2, 'Very High':3,'Extreme':4})

        # Calculate PaymentProgressRatio
        #X['PaymentProgressRatio'] = X['MonthsInRepayment'] / X['OrigLoanTerm']

        # Calculate MonthlyPayment
        p = X['OrigUPB']
        r = X['OrigInterestRate'] / (100 * 12)
        n = X['OrigLoanTerm']
        X['MonthlyPayment'] = round(p * r * ((1 + r) ** n / ((1 + r) ** n - 1)), 2)

        # Calculate CurrentPrincipal
        P = X['OrigUPB']
        r = X['OrigInterestRate'] / (100 * 12)
        E = X['MonthlyPayment']
        #M = X['MonthsInRepayment']
        #X['CurrentPrincipal'] = round((P * (1 + r) ** M) - (E * (((1 + r) ** M - 1) / r)), 2)

        # Calculate TotalAmount
        X['total_amount'] = X['MonthlyPayment'] * X['OrigLoanTerm']

        # Calculate InterestAmount
        X['interest_amount'] = X['total_amount'] - X['OrigUPB']

        # Calculate MonthlyIncome
        X['MonthlyIncome'] = round(X['MonthlyPayment'] / (X['DTI'] / 100), 2)

        X['prepayment']=np.vectorize(prepayment)(X['DTI'],X['MonthlyIncome'])
        X['prepayment']=(X['prepayment']*24)-(X['MonthlyPayment']*24)

        # Drop unneeded columns
        X = X.drop(columns=['MIP','Units', 'FirstTimeHomebuyer','LoanPurpose','SellerName','ServicerName',
                             'Channel','LTV_range','OCLTV_range', 'Credit_range',
                            'Repay_range','DTIBin'])

        # Handle any remaining missing values
        X.replace([np.inf, -np.inf], np.nan, inplace=True)
        X.dropna(inplace=True)

        return X


class PipelineHandler:
    def __init__(self, clf, reg):
        # Initialize the two models
        self.clf = clf
        self.reg = reg
        # Preprocessing pipeline
        self.preprocessing = PreprocessingTransformer()
        self.standarscaler=StandardScaler()
        self.minmaxscaler=MinMaxScaler()
    def fit(self, X, y_class):
        # Preprocess the data
        X_processed = self.preprocessing.transform(X)
        y_class = y_class.loc[X_processed.index].values.ravel()  # Ensure 1D array

        # Select features for classification
        classification_features = ['MonthsDelinquent', 'FICO_Category_Encoded',
                            'YearsInRepayment_Group_Encoded', 'NumBorrowers',
                            'Channel_Frequency', 'LTV_Group_Encoded','OCLTV_Group_Encoded','isHomeFirstBuyer',
                            'OrigInterestRate']

        # Split the data for model training
        X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
        X_processed, y_class, X_processed['prepayment'], test_size=0.2, random_state=42)

        # Ensure that y_class_train and y_class_test are 1D arrays
        y_class_train = y_class_train.ravel()
        y_class_test = y_class_test.ravel()

        X_train_scaled = X_train.copy()
        X_test_scaled = X_test.copy()
        X_train_scaled[classification_features] = self.standarscaler.fit_transform(X_train[classification_features])
        X_test_scaled[classification_features] = self.standarscaler.transform(X_test[classification_features])

        #********************************************************* Classification model
        # Fit classification model
        self.clf.fit(X_train_scaled[classification_features], y_class_train)  # Use scaled data

        # Predict delinquency status
        y_class_pred = self.clf.predict(X_test_scaled[classification_features])  # Use scaled data
        # For evaluation
        # Predict probabilities for ROC-AUC
        y_test_pred_prob = self.clf.predict_proba(X_test_scaled[classification_features])[:, 1]
        # Calculate scores
        testing_accuracy = accuracy_score(y_class_test, y_class_pred)
        roc_auc = roc_auc_score(y_class_test, y_test_pred_prob)
        f1 = f1_score(y_class_test, y_class_pred)
        print("Classification model results:")
        print("testing accuracy:", testing_accuracy)
        print("roc_auc:", roc_auc)
        print("f1_score:", f1)

        #******************************************************* Regression model

        # Select features for regression
        regression_features = ['OrigUPB', 'MonthlyPayment', 'total_amount',
                           'interest_amount', 'MonthlyIncome', 'DTIBins']
        X_train_scaled[regression_features] = self.minmaxscaler.fit_transform(X_train[regression_features])
        X_test_scaled[regression_features] = self.minmaxscaler.transform(X_test[regression_features])

        # Filter delinquent borrowers
        X_filtred_reg_train = X_train_scaled[y_class_train == 0]
        y_reg_filtered_train = y_reg_train[y_class_train == 0]  # Use y_class_train to filter y_reg_train

        X_filtred_reg_test = X_test_scaled[y_class_test == 0]
        y_reg_filtered_test = y_reg_test[y_class_test == 0]    # Use y_class_test to filter y_reg_test

        self.reg.fit(X_filtred_reg_train[regression_features], y_reg_filtered_train)
        # Predict Prepayment Amount
        y_reg_pred = self.reg.predict(X_filtred_reg_test[regression_features])
        # Calculate scores
        # Use mean squared error for regression problems
        b = mean_squared_error(y_reg_filtered_test, y_reg_pred, squared=False)   # RMSE for testing
        c = r2_score(y_reg_filtered_test, y_reg_pred)
        print("Regression model results:")
        print("RMSE", b)
        print("R2", c)

    def predict(self, X):
        # Preprocess the data
        print('X',X)
        X_processed = self.preprocessing.transform(X)
        print('X_processed',X_processed)
        # Select features for classification
        classification_features = ['MonthsDelinquent', 'FICO_Category_Encoded',
                            'YearsInRepayment_Group_Encoded', 'NumBorrowers',
                            'Channel_Frequency', 'LTV_Group_Encoded','OCLTV_Group_Encoded','isHomeFirstBuyer',
                            'OrigInterestRate']
        # Select features for regression model
        regression_features = ['OrigUPB', 'MonthlyPayment', 'total_amount',
                           'interest_amount', 'MonthlyIncome', 'DTIBins']
        # We need to standardize the data
        X_processed[classification_features] = self.standarscaler.transform(X_processed[classification_features])
        # We need to standardize the data
        X_processed[regression_features] = self.minmaxscaler.transform(X_processed[regression_features])

        #*********************Classification model prediction****************************
        # Predict delinquency status
        y_class_pred = self.clf.predict(X_processed[classification_features])

        #***********************Regression model classification**************************
        # Filter data for non-delinquent predictions
        X_filtered_reg_new = X_processed[y_class_pred == 0]

        if X_filtered_reg_new.shape[0] > 0:
        # Predict repayment amount for non-delinquent borrowers
           y_reg_pred_new = self.reg.predict(X_filtered_reg_new[regression_features])
        else:
           y_reg_pred_new = np.array([])  # No non-delinquent borrowers in the new data


        return y_class_pred, y_reg_pred_new



In [ ]:
raw_data_features1.drop(columns=['EverDelinquent'],inplace=True)

<ipython-input-58-a57447dd9d8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_features1.drop(columns=['EverDelinquent'],inplace=True)


In [ ]:
raw_data_features2.drop(columns=['EverDelinquent'],inplace=True)

<ipython-input-59-ed9884ad5cdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_features2.drop(columns=['EverDelinquent'],inplace=True)


In [ ]:
# Training phase
pipeline_handler = PipelineHandler(LogisticRegression(max_iter=1000, solver='lbfgs'), Ridge(alpha=0.1, max_iter=None, solver='sag', tol=0.0001))
pipeline_handler.fit(raw_data_features1, Target1)

Classification model results:
testing accuracy: 1.0
roc_auc: 1.0
f1_score: 1.0
Regression model results:
RMSE 3990.3546720246604
R2 0.9635257628843132


Let's test the predict function

In [ ]:
# Make predictions
y_class_pred_new, y_reg_pred_new = pipeline_handler.predict(raw_data_features2)

# Output the predictions
print("Classification Predictions (Delinquency Status):", y_class_pred_new)
print("Regression Predictions (Repayment Amount for Non-delinquent):", y_reg_pred_new)


X        FirstTimeHomebuyer  MIP  Units  OrigUPB  OrigInterestRate  \
151078                  Y   30      1   196000             6.875   
151079                  Y   18      1    51000             7.375   
151080                  Y   30      1    72000             7.000   
151081                  Y   30      1   173000             7.500   
151082                  Y    0      1    99000             6.875   
...                   ...  ...    ...      ...               ...   
201073                  N    0      1   196000             6.875   
201074                  N    0      1   176000             6.875   
201075                  N   12      1   168000             7.125   
201076                  N   25      1   130000             7.375   
201077                  N    0      1    78000             6.250   

        OrigLoanTerm Channel LoanPurpose NumBorrowers SellerName  \
151078           360       R           P            2         AC   
151079           360       T           P     

As results we  built the pipeline and successfully can fit both models and predict their targets after proceed data  with all Preprocessing steps

In [ ]:
import joblib

# Export the trained pipeline
joblib.dump(pipeline_handler, 'mortgage_pipeline_model_.pkl')

['mortgage_pipeline_model_.pkl']

In [ ]:
print(type(pipeline_handler))

<class '__main__.PipelineHandler'>


In [ ]:
# Load the trained pipeline
loaded_pipeline_handler = joblib.load('mortgage_pipeline_model_.pkl')


In [ ]:
print(type(loaded_pipeline_handler))           # Should be <class '__main__.PipelineHandler'>
print(loaded_pipeline_handler.__dict__)        # Check the internal state


<class '__main__.PipelineHandler'>
{'clf': LogisticRegression(max_iter=1000), 'reg': Ridge(alpha=0.1, solver='sag'), 'preprocessing': PreprocessingTransformer(), 'standarscaler': StandardScaler(), 'minmaxscaler': MinMaxScaler()}
